## HPO Training using Accelerator API

![SpectrumComputeFamily_Conductor-HorizontalColorWhite.png](https://raw.githubusercontent.com/IBM/wmla-learning-path/master/shared-images/hpo.png)

In this notebook you will learn how to submit a model and dataset to the Watson Machine Learning Accelerator (WMLA) API to run Hyper Parameter Optimization (HPO).  

For this notebook you will use a model and dataset that have already been setup to leverage the API.  For details on the API see this link in the Knowledge Center (KC).

[API Documentation](https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/deeplearning.html)

## Imports

To use the WMLA API, we will be using the python requests library

In [1]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import json
import time
import urllib
import pandas as pd
import os,sys
import tarfile
import tempfile
from IPython.display import clear_output
import time
import pprint

# utility print function
def nprint(mystring) :
    print("**{}** : {}".format(sys._getframe(1).f_code.co_name,mystring))

# utility makedir
def makeDirIfNotExist(directory) :
    if not os.path.exists(directory):  
        nprint("Making directory {}".format(directory))
        os.makedirs(directory) 
    else :
        nprint("Directory {} already exists .. ".format(directory))


## Environment details and Project Config:

In [2]:
print("This is your userid...")
!whoami

This is your userid...
vanstee


In [3]:
def getconfig(cfg_in={}):
    cfg = {}
    cfg["master_host"] = 'https://p10a117.pbm.ihost.com'
    cfg["dli_rest_port"] = '9243'
    cfg["sc_rest_port"] = '8643'
    cfg["num_images"] = {"train":200,"valid":20,"test":20}
    # ==== CLASS ENTER User login details below =====
    cfg["wmla_user"] = 'vanstee' # 'b0p036aX'  # <=enter your id here
    cfg["wmla_pwd"] = 'pwvanstee' # 'pwb0p06aX'  # <=enter your pwd here
    # ==== CLASS ENTER User login details above =====
    cfg["sig_name"]  = 'b0p036a-dliauto'
    cfg["code_dir"] = "./code_samples/pytorch_hpo"

    # overwrite configs if passed
    for (k,v) in cfg_in.items() :
        nprint("Overriding Config {}:{} with {}".format(k,cfg[k],v))
        cfg[k] = v
    return cfg

# cfg is used as a global variable throughout this notebook
cfg=getconfig()

In [4]:
# REST call variables
commonHeaders = {'Accept': 'application/json'}

# Use closures for cfg for now ..
def get_tmp_dir() :
    return "./tmp"

def get_tar_file() :
    return get_tmp_dir() + "/" + cfg["wmla_user"]+".modelDir.tar"

#get api endpoint
def get_ep(mode="sc") :
    if mode=="sc" :
        sc_rest_url =  cfg["master_host"] +':'+ cfg["sc_rest_port"] +'/platform/rest/conductor/v1'
        return sc_rest_url
    elif(mode=="dl") :
        dl_rest_url = cfg["master_host"] +':'+cfg["dli_rest_port"] +'/platform/rest/deeplearning/v1'
        return dl_rest_url
    else :
        nprint("Error mode : {} not supported".format(mode))

def myauth():
    return(cfg["wmla_user"],cfg["wmla_pwd"])

print ("SC API Endpoints : {}".format(get_ep("sc")))
print ("DL API Endpoints : {}".format(get_ep("dl")))
print (myauth())
print (get_tar_file())
#myauth = (wmla_user, wmla_pwd)

# Setup Requests session
req = requests.Session()

SC API Endpoints : https://p10a117.pbm.ihost.com:8643/platform/rest/conductor/v1
DL API Endpoints : https://p10a117.pbm.ihost.com:9243/platform/rest/deeplearning/v1
('vanstee', 'pwvanstee')
./tmp/vanstee.modelDir.tar


## Health Check

Check if there is any existing hpo tasks and also verify the platform health

Rest API: **GET platform/rest/deeplearning/v1/hypersearch**
- Description: Get all the hpo task that the login user can access.
- OUTPUT: A list of hpo tasks and each one with the same format which can be found in the api doc.

In [5]:
def hpo_health_check():
    getTuneStatusUrl = get_ep("dl") + '/hypersearch'
    nprint ('getTuneStatusUrl: %s' %getTuneStatusUrl)
    r = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=myauth())
    
    if not r.ok:
        nprint('check hpo task status failed: code=%s, %s'%(r.status_code, r.content))
    else:
        if len(r.json()) == 0:
            nprint('There is no hpo task been created')
        for item in r.json():
            nprint('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))
            #print('Best:%s'%json.dumps(item.get('best'), sort_keys=True, indent=4))

hpo_health_check()


**hpo_health_check** : getTuneStatusUrl: https://p10a117.pbm.ihost.com:9243/platform/rest/deeplearning/v1/hypersearch
**hpo_health_check** : Hpo task: b0p036ac-hpo-1087776628304468, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ab-hpo-1088829220013589, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ae-hpo-1093520538188991, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ai-hpo-1093823185576858, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ad-hpo-1093964072207257, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ac-hpo-1093971889781217, State: FINISHED
**hpo_health_check** : Hpo task: b0p036aa-hpo-1093973673511441, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ah-hpo-1093974201544870, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ag-hpo-1094000147238613, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ae-hpo-1094197964998973, State: FINISHED
**hpo_health_check** : Hpo task: b0p036ac-hpo-1094778911727334, State: FINISHED
**

## Launch an HPO task using API


This part of the lab will step through the steps required to train a simple MNIST (greyscale digits) dataset using WMLA HPO through API ... 

We will be using the **PyTorch** deep learning framework for this example.  Lets take a look at the code ..

* pytorch_mnist_original.py : original sample code from pytorch.org
* pytorch_mnist_HPO.py      : Modified code that implements the required HPO hooks for the WMLA framework
    * Learing rate controlled by HPO
* pytorch_mnist_HPO_advanced.py : Similar example just showing multiple settings 
    * Learning rate controlled by HPO
    * Dropout controlled by HPO

In [6]:
print("Code directory : {}".format(cfg["code_dir"]))
!ls {cfg["code_dir"]}

Code directory : ./code_samples/pytorch_hpo
pytorch_mnist_HPO_advanced.py  pytorch_mnist_original.py
pytorch_mnist_HPO.py	       README.md


### Model file update to Run HPO

**Developer Note** 
The WMLA framework requires 2 changes to your code to support the HPO API, these are :
- Inject hyper-parameters for the sub-training during search
- Retrieve sub-training result metric

We will cover these details in the next 2 sections

For an even more detailed review, see the [documentation in Github](https://github.com/IBM/wmla-learning-path/blob/master/02_hpo_for_developers.md)

#### Model update part 1 - Inject hyper-parameters

The hyper-parameters will be supplied in a file called **config.json** with JSON format,located in the current working directory and can be read direcly as the following example snippet.

<pre>
hyper_params = json.loads(open("<b>config.json</b>").read())
learning_rate = float(hyper_params.get("<b>learning_rate</b>", "0.01"))
</pre>

After this, you can use these hyper-parameters during the model trainings. The **hyper-parameter name** and **value** type is defined through the search space part in body of REST call when launching a new hpo task.

#### Model update part 2 - Retrieve sub-training result metric

At the end of your training run, your code will need to create a file called **val_dict_list.json** with test metrics generated during training. These metrics will be used by the search algorithm to propose new sets of hyper-parameters. Please note that **val_dict_list.json** should be created under the result directory which can be retrieved through the environment variable **RESULT_DIR**.

<pre>
with open('{}/val_dict_list.json'.format(os.environ['<b>RESULT_DIR</b>']), 'w') as f:
    json.dump(test_metrics, f)
</pre>

The content of **val_dict_list.json** will be some thing as below, **step** is some thing optional meaning the training iteration or epochs, one of **loss** and **accuracy** can be the name of target metric to optimize, at least one metric need to be included here. The specific name of metric used to optimize (minimize or maximize) is defined in the body of REST call when launching a new hpo task. 

```
[
{‘step’: 1, ‘loss’:0.2487, ‘accuracy’: 0.4523},
{‘step’: 2, ‘loss’:0.1487, ‘accuracy’: 0.5523},
{‘step’: 3, ‘loss’:0.1087, ‘accuracy’: 0.6523},
…
]
```

**We have added this to the pytorch_mnist_HPO.py file already for you!**

In [7]:
# !grep -A1 -B2 val_dict {cfg["code_dir"]}/pytorch_mnist_HPO.py

### Launch HPO task

Here we package up our model to send to the API for HPO.  Lets see how this works ...



REST API: **POST /platform/rest/deeplearning/v1/hypersearch**
- Description: Start a new HPO task
- Content-type: Multi-Form
- Multi-Form Data:
  - files: Model files tar package, ending with `.modelDir.tar`
  - form-filed: {‘data’: ‘String format of input parameters to start hpo task, let’s call it as **hpo_input** and show its specification later’}


#### Package model files for training

Package the updated model files into a tar file ending with `.modelDir.tar`

REST API expects a modelDir.tar with the model code inside ..


In [8]:
# Tar up 
def make_tarfile():
    makeDirIfNotExist(get_tmp_dir())
    tar_archive_base=os.path.basename(cfg["code_dir"])
    nprint("Tarring up {} to {}".format(cfg["code_dir"],get_tmp_dir()))
    nprint("Adding base directory to archive : {}".format(tar_archive_base))
    with tarfile.open(get_tar_file(), "w:gz") as tar:
        tar.add(cfg["code_dir"], arcname=tar_archive_base)
#MODEL_DIR_SUFFIX = ".modelDir.tar"
#tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)

make_tarfile()

files = {'file': open(get_tar_file(), 'rb')}
print("Files : {}".format(files))

**makeDirIfNotExist** : Making directory ./tmp
**make_tarfile** : Tarring up ./code_samples/pytorch_hpo to ./tmp
**make_tarfile** : Adding base directory to archive : pytorch_hpo
Files : {'file': <_io.BufferedReader name='./tmp/vanstee.modelDir.tar'>}


#### Construct POST request data

**hpo_input** will be a Python dict or json format as below, convert to string when calling REST.

In [9]:
# Note, this 
data =  {
        'modelSpec': # Define the model training related parameters
        {
            # Spark instance group which will be used to run the HPO sub-trainings. The Spark instance group selected
            # here should match the sub-training args, for example, if the sub-training args try to run a EDT job,
            # then we should put a Spark instance group with capability to run EDT job here.
            'sigName': cfg["sig_name"],

            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            #
            'args': '--exec-start PyTorch --cs-datastore-meta type=fs --python-version 3.6\
                     --gpuPerWorker 1 --model-main pytorch_mnist_HPO.py --model-dir pytorch_hpo\
                     --debug-level debug'
                
        },
    
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband, ExperimentGridSearch
            'algorithm': 'Random', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': 60,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 4,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 2, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.
        'hyperParams':
        [
             {
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'learning_rate',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'DOUBLE',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.001,
                 'maxDbVal': 0.1,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 0,
                 'maxIntVal': 0,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': []
                 #step size to split the Range space. ONLY valid when type is Range
                 #'step': '0.002',
             }
         ]
    }
mydata={'data':json.dumps(data)}

#### Submit the Post request

Submit hpo task through the Post call and a hpo name/id as string format will get back.

**Note**:This cannot be submitted twice.. you need to rebuild the tar file prior to resubmitting

In [10]:
def submit_job(job_dict,job_files,job_auth):
    startTuneUrl=get_ep('dl') + '/hypersearch'
    nprint("startTuneUrl : {}".format(startTuneUrl))
    nprint("files : {}".format(job_files))
    nprint("myauth() : {}".format(job_auth))
    #print("hpo_job_id : {}".format(hpo_job_id))
    r = req.post(startTuneUrl, headers=commonHeaders, data=job_dict, files=job_files, verify=False, auth=job_auth)
    hpo_name=None
    if r.ok:
        hpo_name = r.json()
        print ('\nModel submitted successfully: {}'.format(hpo_name))
        
    else:
        print('\nModel submission failed with code={}, {}'. format(r.status_code, r.content))
    return hpo_name

hpo_job_id = submit_job(mydata,files,myauth())
print("hpo_job_id : {}".format(hpo_job_id))

**submit_job** : startTuneUrl : https://p10a117.pbm.ihost.com:9243/platform/rest/deeplearning/v1/hypersearch
**submit_job** : files : {'file': <_io.BufferedReader name='./tmp/vanstee.modelDir.tar'>}
**submit_job** : myauth() : ('vanstee', 'pwvanstee')

Model submitted successfully: vanstee-hpo-1976537835784211
hpo_job_id : vanstee-hpo-1976537835784211


### Query Status until complete

In [ ]:
def query_job_status(job_id,refresh_rate=3) :

    getHpoUrl = get_ep('dl') +'/hypersearch/'+ job_id
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=myauth())
        experiments=res.json()['experiments']
        experiments = pd.DataFrame.from_dict(experiments)
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(experiments)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
query_job_status(hpo_job_id,refresh_rate=10)

Refreshing every 10 seconds


,appId,driverId,endTime,hyperParams,id,maxiteration,metricVal,startTime,state
0,vanstee-1976539608359161-1603577529,driver-20200624113238-0213-51d43486-68ff-44b2-9139-2a0e0145abe8,2020-06-24 11:37:53,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.07537755628607677'}]",0,0,0.044015,2020-06-24 11:32:38,FINISHED
1,vanstee-1976545663608356-867777385,driver-20200624113244-0214-1f29e121-42a6-4e44-82f6-05945527c4fc,2020-06-24 11:37:54,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.020268239667235212'}]",1,0,0.097599,2020-06-24 11:32:44,FINISHED
2,vanstee-1976857795655493-205369088,driver-20200624113756-0215-18a93221-e744-4869-abc3-e74d6b87c6d7,NaN,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.06521759926157743'}]",2,0,9999.990000,2020-06-24 11:37:56,RUNNING
3,vanstee-1976863945276560-2047140053,driver-20200624113802-0216-7053f35d-1462-4046-b402-556271523311,NaN,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.07959776988962579'}]",3,0,9999.990000,2020-06-24 11:38:02,RUNNING


{ 'best': { 'appId': 'vanstee-1976539608359161-1603577529',
            'driverId': 'driver-20200624113238-0213-51d43486-68ff-44b2-9139-2a0e0145abe8',
            'endTime': '2020-06-24 11:37:53',
            'hyperParams': [ { 'dataType': 'double',
                               'fixedVal': '0.07537755628607677',
                               'name': 'learning_rate',
                               'userDefined': False}],
            'id': 0,
            'maxiteration': 0,
            'metricVal': 0.04401541900634766,
            'startTime': '2020-06-24 11:32:38',
            'state': 'FINISHED'},
  'complete': 2,
  'createtime': '2020-06-24 11:32:35',
  'creator': 'vanstee',
  'duration': '00:07:36',
  'experiments': [ { 'appId': 'vanstee-1976539608359161-1603577529',
                     'driverId': 'driver-20200624113238-0213-51d43486-68ff-44b2-9139-2a0e0145abe8',
                     'endTime': '2020-06-24 11:37:53',
                     'hyperParams': [ { 'dataType': 'double',
 

### Show the Best Result of HPO Job

In [ ]:
# Lets query our result to see what happened during HPO training!

#res.ok
#res.json()
#print(type(res))
#print(dir(res))
#print(json.dumps(res.json(), indent=4, sort_keys=True))
        
print('Hpo task %s completes with state %s'%(hpo_job_id, res.json()['state']))
print("Best HPO result ...")
res.json()["best"]


#### Notebook Complete 
Congratulations, you have completed our demonstration of using WMLA for distributed hyperparameter optimization search

## [Optional Advanced Example] : Random Optimization using Multiple Parameters

In [66]:
# Note, this 
data =  {
        'modelSpec': # Define the model training related parameters
        {
            # Spark instance group which will be used to run the HPO sub-trainings. The Spark instance group selected
            # here should match the sub-training args, for example, if the sub-training args try to run a EDT job,
            # then we should put a Spark instance group with capability to run EDT job here.
            'sigName': cfg["sig_name"],

            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            #
            'args': '--exec-start PyTorch --cs-datastore-meta type=fs --python-version 3.6\
                     --gpuPerWorker 1 --model-main pytorch_mnist_HPO_advanced.py --model-dir pytorch_hpo\
                     --debug-level debug'
                
        },
    
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband, ExperimentGridSearch
            'algorithm': 'Random', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': 60,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 60,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 30, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.
        'hyperParams':
        [
             {
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'learning_rate',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'DOUBLE',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.0001,
                 'maxDbVal': 0.1,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 0,
                 'maxIntVal': 0,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': []
                 #step size to split the Range space. ONLY valid when type is Range
                 #'step': '0.002',
             },{
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'num_hidden_layers',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'INT',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.0,
                 'maxDbVal': 0.0,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 25,
                 'maxIntVal': 200,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': []
                 #step size to split the Range space. ONLY valid when type is Range
                 #'step': '0.002',
             },             {
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'dropout_rate',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'DOUBLE',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.00,
                 'maxDbVal': 0.40,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 0,
                 'maxIntVal': 0,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': []
                 #step size to split the Range space. ONLY valid when type is Range
                 #'step': '0.002',
             }
         ]
    }
mydata={'data':json.dumps(data)}

### Submit Advanced Job

In [67]:
files = {'file': open(get_tar_file(), 'rb')}
print("Files : {}".format(files))
hpo_job_id = submit_job(mydata,files,myauth())


Files : {'file': <_io.BufferedReader name='/gpfs/home/s4s004/vanstee/2020-05-wmla/tmp/vanstee.modelDir.tar'>}
**submit_job** : startTuneUrl : https://p10a117.pbm.ihost.com:9243/platform/rest/deeplearning/v1/hypersearch
**submit_job** : files : {'file': <_io.BufferedReader name='/gpfs/home/s4s004/vanstee/2020-05-wmla/tmp/vanstee.modelDir.tar'>}
**submit_job** : myauth() : ('vanstee', 'pwvanstee')

Model submitted successfully: vanstee-hpo-336094667871905


In [72]:
res = query_job_status("vanstee-hpo-328047362438688",refresh_rate=25)

Refreshing every 25 seconds


,id,hyperParams,state,metricVal,maxiteration,appId,driverId,startTime,endTime
0,0,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.09361823202572035'}, {'name': ...",FINISHED,0.047327,0,vanstee-328061834544208-1045117375,driver-20200605093800-0076-49bda2b5-44ee-4e8b-9b0e-2782c45aec4a,2020-06-05 09:38:00,2020-06-05 09:43:30
1,1,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.07096787497253654'}, {'name': ...",FINISHED,0.043178,0,vanstee-328067937657942-1504271243,driver-20200605093806-0077-dfab81b7-1729-40bc-8924-ceaa729e7d56,2020-06-05 09:38:06,2020-06-05 09:43:44
2,2,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.05554085409528766'}, {'name': ...",FINISHED,0.043657,0,vanstee-328074022276158-1353756732,driver-20200605093812-0078-d127292b-2747-464f-84ce-031a9890e016,2020-06-05 09:38:12,2020-06-05 09:43:35
3,3,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.058858649617074696'}, {'name':...",FINISHED,0.040554,0,vanstee-328080178892141-1730704806,driver-20200605093818-0079-c6453424-958d-491e-9459-387dc1ed4a9b,2020-06-05 09:38:19,2020-06-05 09:43:39
4,4,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.07743818851387108'}, {'name': ...",FINISHED,0.033538,0,vanstee-328086375747885-79870233,driver-20200605093824-0080-936f605a-8f0d-4de4-b26b-0289372d45a7,2020-06-05 09:38:25,2020-06-05 09:43:31
5,5,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.07434147405463533'}, {'name': ...",FINISHED,0.033083,0,vanstee-328092401164469-177859784,driver-20200605093831-0081-4dccc601-a3c6-4d5d-b4d2-bed65df81587,2020-06-05 09:38:31,2020-06-05 09:43:32
6,6,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.08863006608234658'}, {'name': ...",FINISHED,0.033722,0,vanstee-328098585195601-1810560942,driver-20200605093837-0082-ae6b70ed-b4cd-4ef2-ad49-dfa35006299a,2020-06-05 09:38:37,2020-06-05 09:43:36
7,7,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.0034994359228758235'}, {'name'...",FINISHED,0.229928,0,vanstee-328104677160421-1000976594,driver-20200605093843-0083-dd1e9ab2-c19c-4c73-a205-1040f313e033,2020-06-05 09:38:43,2020-06-05 09:43:33
8,8,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.0336173920854255'}, {'name': '...",FINISHED,0.047850,0,vanstee-328110709022615-268348752,driver-20200605093849-0084-454a143d-1f09-426e-96cb-c4038147cd16,2020-06-05 09:38:49,2020-06-05 09:43:37
9,9,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.08056522947257845'}, {'name': ...",FINISHED,0.032547,0,vanstee-328116961108660-1910171794,driver-20200605093855-0085-6d5c45e5-b9a3-42a2-ab58-ed620858c6ce,2020-06-05 09:38:55,2020-06-05 09:43:38


{ 'best': { 'appId': 'vanstee-328216516134174-1721691813',
            'driverId': 'driver-20200605094035-0101-4257739b-1a94-44f1-b58f-0752dbafd7f7',
            'endTime': '2020-06-05 09:48:03',
            'hyperParams': [ { 'dataType': 'double',
                               'fixedVal': '0.09990502970155099',
                               'name': 'learning_rate',
                               'userDefined': False},
                             { 'dataType': 'int',
                               'fixedVal': '199',
                               'name': 'num_hidden_layers',
                               'userDefined': False},
                             { 'dataType': 'double',
                               'fixedVal': '0.154413970850679',
                               'name': 'dropout_rate',
                               'userDefined': False}],
            'id': 25,
            'maxiteration': 0,
            'metricVal': 0.028594832801818848,
            'startTime': '2020-06-

                                        'userDefined': False}],
                     'id': 9,
                     'maxiteration': 0,
                     'metricVal': 0.03254700603485108,
                     'startTime': '2020-06-05 09:38:55',
                     'state': 'FINISHED'},
                   { 'appId': 'vanstee-328123106761742-1719830834',
                     'driverId': 'driver-20200605093901-0086-19ecbae9-4fe7-4cc5-bad5-3fd4377ba46e',
                     'endTime': '2020-06-05 09:44:45',
                     'hyperParams': [ { 'dataType': 'double',
                                        'fixedVal': '0.08657071881490906',
                                        'name': 'learning_rate',
                                        'userDefined': False},
                                      { 'dataType': 'int',
                                        'fixedVal': '124',
                                        'name': 'num_hidden_layers',
                                    

                     'endTime': '2020-06-05 09:55:11',
                     'hyperParams': [ { 'dataType': 'double',
                                        'fixedVal': '0.0706673061609298',
                                        'name': 'learning_rate',
                                        'userDefined': False},
                                      { 'dataType': 'int',
                                        'fixedVal': '99',
                                        'name': 'num_hidden_layers',
                                        'userDefined': False},
                                      { 'dataType': 'double',
                                        'fixedVal': '0.25989556761705246',
                                        'name': 'dropout_rate',
                                        'userDefined': False}],
                     'id': 41,
                     'maxiteration': 0,
                     'metricVal': 0.04080289249420166,
                     'startTime': '2020-0

### Best Result 

In [73]:
print('Hpo task %s completes with state %s'%(hpo_job_id, res.json()['state']))
print("Best HPO result ...")
res.json()["best"]

AttributeError: 'NoneType' object has no attribute 'json'